In [2]:
from __future__ import division, print_function
import sklearn
import pandas as pd
import numpy as np
import os
import import_ipynb

from my_libs.df_converter import lowerstr
from my_libs.calc_features import make_prepared
import my_libs.data_paths as dp

# from df_converter import lowerstr
# from calc_features import make_prepared
# import data_paths as dp
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

importing Jupyter notebook from df_converter.ipynb
importing Jupyter notebook from calc_features.ipynb
importing Jupyter notebook from data_paths.ipynb


In [15]:
chem = {
    'Mn': 0,  
    'P': 0,
    'Cr': 0,
    'V': 0, 
    'Mo': 0, 
    'Ti': 0, 
    'Nb': 0, 
    'N': 0.004, 
    'S': 0}

In [16]:
def create_prepared_data(general_modes_chem_filename=dp.general_modes_chem,
                         valid_modes_chem_filename=dp.valid_modes_chem,
                         exp_filename=dp.scretching,
                         prepared_general_filename=dp.general_prepared,
                         prepared_valid_filename=dp.valid_prepared,
                         ys_type = None
                        ):
    """Из трех файлов (два с режимами основной и валидация, и третий с испытанием) формируем два файла 
    с данными подготовленными для формирования датасета 
    
    Keyword arguments:
    general_modes_chem_filename, valid_modes_chem_filename -  имена файлов с режимами и химией (лежать должны в папке с промежуточными файлами)
    
    exp_filename - имя файла с испытаниями (лежать должен в папке с промежуточными файлами)
     
    prepared_general_filename, prepared_valid_filename -  имена файлов для сохранения (создаются в папке с промежуточными файлами)
    
    ys_type -- тип предела текучести по котрому отделять датафрейм (по умолчанию возвращает со всеми типами) (число, если нужно вернуть все то None) 
    
    
    Возвращает: [general_df, valid_df] список из двух датафреймов, которые сохраняются в prepared файлы (сначала общий, потом валид) 
    
    """
    
    general_df = pd.read_excel(dp.between_data_path+general_modes_chem_filename)
    valid_df = pd.read_excel(dp.between_data_path+valid_modes_chem_filename)
    exp_df = pd.read_excel(dp.between_data_path+exp_filename)
    exp_df = lowerstr(exp_df)
    valid_df['№ партии'] = valid_df['№ партии'].apply(lambda x: str(x).replace('.0', ''))
    general_df = make_prepared(general_df, exp_df, chem)
    valid_df = make_prepared(valid_df, exp_df, chem)  
    if ys_type!= None:
        general_df = general_df[general_df['Тип предела текучести (1186)']==ys_type]
        valid_df = valid_df[valid_df['Тип предела текучести (1186)']==ys_type]
        print('shapы после отсечения по типу текучести', general_df.shape, valid_df.shape)
    general_df.to_excel(dp.between_data_path+prepared_general_filename)
    valid_df.to_excel(dp.between_data_path+prepared_valid_filename)
    return [general_df, valid_df]